In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [9]:
!ls

data


In [0]:
import os
os.chdir('./drive/My Drive/Signature_Forgery')

In [0]:
import cv2                 
import numpy as np         
import os                  
from random import shuffle 
from tqdm import tqdm 
import numpy as np

from keras.utils import to_categorical
import matplotlib.pyplot as plt

import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

from sklearn.metrics import accuracy_score

In [0]:
TRAIN_DIR = 'data/train_set'
TEST_DIR = 'data/test_set'
IMG_SIZE = 28

In [0]:
def label_img(img):
    image_label = img.split('.')
    name = image_label[0]
    # checking if the signature is forged or genuine
    #                            
    
    if name[4:7] == name[-3:]:
        return 1
  
    elif name[4:7] != name[-3:]:
        return 0

def person_img(img):
    label = img.split('.')
    k = label[0]
    return [k[-3:]]


def create_features(DIR):
    features = []
    labels = []
    for img in tqdm(os.listdir(DIR)):
        label = label_img(img)
        person = person_img(img)
       
        path = os.path.join(DIR,img)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        features.append([img])
        labels.append(np.array(label))
    #shuffle(training_data)
    #np.save('train_data.npy', training_data)
    return features

def create_labels(DIR):
    labels = []
    for img in tqdm(os.listdir(DIR)):
        label = label_img(img)
        labels.append(np.array(label))
    return labels

In [15]:
train_features  = np.array(create_features(TRAIN_DIR))
test_features  = np.array(create_features(TEST_DIR))
train_features_reshaped = train_features.reshape(-1, 28,28, 1)
test_features_reshaped = test_features.reshape(-1, 28,28, 1)

train_features_reshaped = train_features_reshaped.astype('float32')
test_features_reshaped = test_features_reshaped.astype('float32')
train_features_reshaped = train_features_reshaped / 255.
test_features_reshaped = test_features_reshaped / 255.

train_labels = np.array(create_labels(TRAIN_DIR))
test_labels = np.array(create_labels(TEST_DIR))


train_labels_one_hot = to_categorical(train_labels)
test_labels_one_hot = to_categorical(test_labels)

100%|██████████| 121/121 [00:00<00:00, 92241.15it/s]


In [17]:
print(test_labels_one_hot.shape)

(121, 2)


In [0]:
batch_size = 64
epochs = 100
num_classes = 2

In [50]:
model = Sequential()
model.add(Conv2D(64,kernel_size=(3,3),activation = 'linear',input_shape=(28,28,1),padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D((2,2),padding='same'))
model.add(Conv2D(32,kernel_size=(3,3),activation = 'linear',padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D((2,2),padding='same'))
model.add(Flatten())
model.add(Dense(1000, activation='linear'))
model.add(LeakyReLU(alpha=0.1))   
model.add(Dense(100, activation='linear'))
model.add(LeakyReLU(alpha=0.1))   
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 28, 28, 64)        640       
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 28, 28, 64)        0         
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 14, 14, 32)        18464     
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 14, 14, 32)        0         
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 7, 7, 32)          0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 1568)              0         
__________

In [51]:

fit = model.fit(train_features_reshaped, train_labels_one_hot, batch_size=batch_size,epochs=epochs,verbose=1,validation_split=0.3)

Train on 125 samples, validate on 54 samples
Epoch 1/100
125/125 [==============================] - 2s 12ms/step - loss: 0.8588 - acc: 0.5520 - val_loss: 0.8092 - val_acc: 0.3889
Epoch 2/100
125/125 [==============================] - 0s 2ms/step - loss: 0.6877 - acc: 0.5520 - val_loss: 0.6760 - val_acc: 0.6111
Epoch 3/100
125/125 [==============================] - 0s 2ms/step - loss: 0.7039 - acc: 0.4480 - val_loss: 0.6851 - val_acc: 0.6111
Epoch 4/100
125/125 [==============================] - 0s 2ms/step - loss: 0.6925 - acc: 0.4480 - val_loss: 0.7042 - val_acc: 0.3889
Epoch 5/100
125/125 [==============================] - 0s 3ms/step - loss: 0.6962 - acc: 0.5520 - val_loss: 0.7217 - val_acc: 0.3889
Epoch 6/100
125/125 [==============================] - 0s 3ms/step - loss: 0.6850 - acc: 0.5520 - val_loss: 0.7116 - val_acc: 0.3889
Epoch 7/100
125/125 [==============================] - 0s 3ms/step - loss: 0.6868 - acc: 0.5520 - val_loss: 0.7220 - val_acc: 0.3889
Epoch 8/100
125/125 [==

In [52]:

score, acc = model.evaluate(test_features_reshaped, test_labels_one_hot,
                            batch_size=batch_size)

121/121 [==============================] - 0s 684us/step


In [53]:
print(score,acc)

1.7043763014895856 0.6776859385908143
